In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.preprocessing import OneHotEncoder

In [94]:
df = pd.read_csv("AB_NYC_2019.csv")
df = df.rename(columns={'neighbourhood_group': 'nghb_group', 'neighbourhood': 'nghb'})
df = df.drop(columns=['id', 'name', 'host_id', 'last_review', 'availability_365'])
df.isna().sum()

host_name                            21
nghb_group                            0
nghb                                  0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
dtype: int64

Replacing reviews_per_month NaN with 0, as these are NaNs, because with 0 reviews division gives infinity, hence it can easily be replaced with 0. Then, removing all NaN rows, as only left NaN are 21 in host_name column

In [95]:
df['reviews_per_month'] = df['reviews_per_month'].replace(np.nan, 0)
df = df.dropna()

In [99]:
nghb_group = df[['nghb_group']]
nghb_group_encoder = OneHotEncoder()
nghb_group_1hot = nghb_group_encoder.fit_transform(nghb_group)

nghb = df[['nghb']]
nghb_encoder = OneHotEncoder()
nghb_1hot = nghb_encoder.fit_transform(nghb)


host_name = df[['host_name']]
hots_name_encoder = OneHotEncoder()
host_name_encoder_1hot = hots_name_encoder.fit_transform(host_name)

room_type = df[['room_type']]
room_type_encoder = OneHotEncoder()
room_type_encoder_1hot = room_type_encoder.fit_transform(room_type)

pd.DataFrame(nghb_1hot.toarray(), columns=nghb_encoder.categories_)

,Allerton,Arden Heights,Arrochar,Arverne,Astoria,Bath Beach,Battery Park City,Bay Ridge,Bay Terrace,"Bay Terrace, Staten Island",...,Westerleigh,Whitestone,Williamsbridge,Williamsburg,Willowbrook,Windsor Terrace,Woodhaven,Woodlawn,Woodrow,Woodside
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
pd.isna(df['host_name']).sum()

0

In [91]:
df.head()

,host_name,nghb_group,nghb,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count
0,"(0, 4989)\t1.0\n (1, 4785)\t1.0\n (2, 2909...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",40.64749,-73.97237,"(0, 1)\t1.0\n (1, 0)\t1.0\n (2, 1)\t1.0\n ...",149,1,9,0.21,6
1,"(0, 4989)\t1.0\n (1, 4785)\t1.0\n (2, 2909...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",40.75362,-73.98377,"(0, 1)\t1.0\n (1, 0)\t1.0\n (2, 1)\t1.0\n ...",225,1,45,0.38,2
2,"(0, 4989)\t1.0\n (1, 4785)\t1.0\n (2, 2909...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",40.80902,-73.94190,"(0, 1)\t1.0\n (1, 0)\t1.0\n (2, 1)\t1.0\n ...",150,3,0,0.00,1
3,"(0, 4989)\t1.0\n (1, 4785)\t1.0\n (2, 2909...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",40.68514,-73.95976,"(0, 1)\t1.0\n (1, 0)\t1.0\n (2, 1)\t1.0\n ...",89,1,270,4.64,1
4,"(0, 4989)\t1.0\n (1, 4785)\t1.0\n (2, 2909...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",40.79851,-73.94399,"(0, 1)\t1.0\n (1, 0)\t1.0\n (2, 1)\t1.0\n ...",80,10,9,0.10,1
